# Ltlcross_runner tool settings

In [1]:
def ltl_to_ba(tgba=False):
    spot_n = 'Spot'
    spot_o = '' if tgba else ' -B'
    ltl3ba_n = 'LTL3BA'
    ltl3ba_o = ' -H2' if tgba else ' -H3' 
    ba_suff = '.TGBA' if tgba else '.NBA'
    tools = {
        '{}{}'.format(spot_n,ba_suff) :  \
            'ltl2tgba{} -f %f'.format(spot_o),
        '{}{}'.format(ltl3ba_n,ba_suff) :  \
            'ltl3ba -M0{} -f %s'.format(ltl3ba_o),
        '{}d{}'.format(ltl3ba_n,ba_suff) :  \
            'ltl3ba -M1{} -f %s'.format(ltl3ba_o),
    }
    if tgba:
        tools['LTL3TELA'] = 'ltl3tela -f %f'
    return tools

In [2]:
ltl_to_ba(True)

{'LTL3BA.TGBA': 'ltl3ba -M0 -H2 -f %s',
 'LTL3BAd.TGBA': 'ltl3ba -M1 -H2 -f %s',
 'LTL3TELA': 'ltl3tela -f %f',
 'Spot.TGBA': 'ltl2tgba -f %f'}

In [3]:
ltl_to_ba()

{'LTL3BA.NBA': 'ltl3ba -M0 -H3 -f %s',
 'LTL3BAd.NBA': 'ltl3ba -M1 -H3 -f %s',
 'Spot.NBA': 'ltl2tgba -B -f %f'}

In [4]:
def det_pair(det_tool, ltl_to_ba, dstar=False):
    '''produces a dictionary that combines tool for
    determinization with tools for LTL->BA translation
    
    Parameters
    ----------
    `det_tool`  : a pair of `name` and `command`
    `ltl_to_ba` : a `dict` of name->command
    `dstar`     : `Bool` that indicates whether to use ltl2dstar interface
    '''
    if not dstar:
        tools = {
            '{}+{}'.format(det_tool[0],name) :
                '{} | {} > %O'.format(cmd,det_tool[1]) for
                    name, cmd in ltl_to_ba.items()
        }
    else:
        tools = {
            '{}+{}'.format(det_tool[0],name) :
                '{} -t "{} > %%H" %L %O'.format(det_tool[1],cmd.replace('%s','%%s').replace('%f','%%s')) for
                    name, cmd in ltl_to_ba.items()
    }
    return tools

In [5]:
# name, command, generalized, dstar_interface
determinization_tools = [
    ('Spot','autfilt -DG',False, False),
    ('Spot','autfilt -DG',True, False),
    ('ltl2dstar','ltl2dstar -H',False, True),
    ('ltl2dstar(NBA)','ltl2dstar -B -H - -',False, False)
]
full_tools = {}
for tool in determinization_tools:
    full_tools.update(det_pair((tool[0],tool[1]),ltl_to_ba(tool[2]),tool[3]))

In [6]:
full_tools

{'Spot+LTL3BA.NBA': 'ltl3ba -M0 -H3 -f %s | autfilt -DG > %O',
 'Spot+LTL3BA.TGBA': 'ltl3ba -M0 -H2 -f %s | autfilt -DG > %O',
 'Spot+LTL3BAd.NBA': 'ltl3ba -M1 -H3 -f %s | autfilt -DG > %O',
 'Spot+LTL3BAd.TGBA': 'ltl3ba -M1 -H2 -f %s | autfilt -DG > %O',
 'Spot+LTL3TELA': 'ltl3tela -f %f | autfilt -DG > %O',
 'Spot+Spot.NBA': 'ltl2tgba -B -f %f | autfilt -DG > %O',
 'Spot+Spot.TGBA': 'ltl2tgba -f %f | autfilt -DG > %O',
 'ltl2dstar(NBA)+LTL3BA.NBA': 'ltl3ba -M0 -H3 -f %s | ltl2dstar -B -H - - > %O',
 'ltl2dstar(NBA)+LTL3BAd.NBA': 'ltl3ba -M1 -H3 -f %s | ltl2dstar -B -H - - > %O',
 'ltl2dstar(NBA)+Spot.NBA': 'ltl2tgba -B -f %f | ltl2dstar -B -H - - > %O',
 'ltl2dstar+LTL3BA.NBA': 'ltl2dstar -H -t "ltl3ba -M0 -H3 -f %%s > %%H" %L %O',
 'ltl2dstar+LTL3BAd.NBA': 'ltl2dstar -H -t "ltl3ba -M1 -H3 -f %%s > %%H" %L %O',
 'ltl2dstar+Spot.NBA': 'ltl2dstar -H -t "ltl2tgba -B -f %%s > %%H" %L %O'}

In [7]:
rab4 = 'Rab4/bin/'
sacc = ' | autfilt -S > %O'
rabinizers = {
    "R3"      : 'java -jar Rab3/rabinizer3.1.jar -silent -format=hoa -out=std %[eiRWM]f > %O',
    "R3.DSRA" : 'java -jar Rab3/rabinizer3.1.jar -silent -format=hoa -out=std -auto=sr %[eiRWM]f > %O',
    "R4"      : rab4 + 'ltl2dgra %f > %O',
    "R4.DSRA" : rab4 + 'ltl2dra %f' + sacc
}
ltl3dra = {
    "LTL3DRA" : 'ltl3dra -f %s > %O',
    "LTL3DRA.DSRA" : 'ltl3dra -H3 -f %s > %O'
}
ltl2tgba = {
    "ltl2tgba"      : 'ltl2tgba -DG -f %f > %O',
}
parity = {
    "ltl2dpa(ldba)"      : rab4 + 'ltl2dpa --mode=ldba %f > %O',
    "ltl2dpa(Rab)"       : rab4 + 'ltl2dpa --mode=rabinizer %f > %O',
}

In [8]:
full_tools.update(rabinizers)
full_tools.update(ltl2tgba)
full_tools.update(parity)

In [9]:
ltl3dra_tools = full_tools.copy()

In [10]:
ltl3dra_tools.update(ltl3dra)

In [11]:
ltl3dra_tools

{'LTL3DRA': 'ltl3dra -f %s > %O',
 'LTL3DRA.DSRA': 'ltl3dra -H3 -f %s > %O',
 'R3': 'java -jar Rab3/rabinizer3.1.jar -silent -format=hoa -out=std %[RWM]f > %O',
 'R3.DSRA': 'java -jar Rab3/rabinizer3.1.jar -silent -format=hoa -out=std -auto=sr %[RWM]f > %O',
 'R4': 'Rab4/bin/ltl2dgra %f > %O',
 'R4.DSRA': 'Rab4/bin/ltl2dra %f | autfilt -S > %O',
 'Spot+LTL3BA.NBA': 'ltl3ba -M0 -H3 -f %s | autfilt -DG > %O',
 'Spot+LTL3BA.TGBA': 'ltl3ba -M0 -H2 -f %s | autfilt -DG > %O',
 'Spot+LTL3BAd.NBA': 'ltl3ba -M1 -H3 -f %s | autfilt -DG > %O',
 'Spot+LTL3BAd.TGBA': 'ltl3ba -M1 -H2 -f %s | autfilt -DG > %O',
 'Spot+LTL3TELA': 'ltl3tela -f %f | autfilt -DG > %O',
 'Spot+Spot.NBA': 'ltl2tgba -B -f %f | autfilt -DG > %O',
 'Spot+Spot.TGBA': 'ltl2tgba -f %f | autfilt -DG > %O',
 'ltl2dpa(Rab)': 'Rab4/bin/ltl2dpa --mode=rabinizer %f > %O',
 'ltl2dpa(ldba)': 'Rab4/bin/ltl2dpa --mode=ldba %f > %O',
 'ltl2dstar(NBA)+LTL3BA.NBA': 'ltl3ba -M0 -H3 -f %s | ltl2dstar -B -H - - > %O',
 'ltl2dstar(NBA)+LTL3BAd.N

In [12]:
full_tools

{'R3': 'java -jar Rab3/rabinizer3.1.jar -silent -format=hoa -out=std %[RWM]f > %O',
 'R3.DSRA': 'java -jar Rab3/rabinizer3.1.jar -silent -format=hoa -out=std -auto=sr %[RWM]f > %O',
 'R4': 'Rab4/bin/ltl2dgra %f > %O',
 'R4.DSRA': 'Rab4/bin/ltl2dra %f | autfilt -S > %O',
 'Spot+LTL3BA.NBA': 'ltl3ba -M0 -H3 -f %s | autfilt -DG > %O',
 'Spot+LTL3BA.TGBA': 'ltl3ba -M0 -H2 -f %s | autfilt -DG > %O',
 'Spot+LTL3BAd.NBA': 'ltl3ba -M1 -H3 -f %s | autfilt -DG > %O',
 'Spot+LTL3BAd.TGBA': 'ltl3ba -M1 -H2 -f %s | autfilt -DG > %O',
 'Spot+LTL3TELA': 'ltl3tela -f %f | autfilt -DG > %O',
 'Spot+Spot.NBA': 'ltl2tgba -B -f %f | autfilt -DG > %O',
 'Spot+Spot.TGBA': 'ltl2tgba -f %f | autfilt -DG > %O',
 'ltl2dpa(Rab)': 'Rab4/bin/ltl2dpa --mode=rabinizer %f > %O',
 'ltl2dpa(ldba)': 'Rab4/bin/ltl2dpa --mode=ldba %f > %O',
 'ltl2dstar(NBA)+LTL3BA.NBA': 'ltl3ba -M0 -H3 -f %s | ltl2dstar -B -H - - > %O',
 'ltl2dstar(NBA)+LTL3BAd.NBA': 'ltl3ba -M1 -H3 -f %s | ltl2dstar -B -H - - > %O',
 'ltl2dstar(NBA)+Spot

In [27]:
def mint(s):
    return("\\mintinline{bash}{"+s+"}")

In [30]:
def latex(d):
    for name, cmd in sorted(d.items()):
        print("    & {}\n    & {}\\\\".format(name,mint(cmd)))